This script reads EEG electrode coordinate files for multiple participants and visualizes their positions
in a 3D space using Matplotlib. Each participant's electrode montage is displayed in a unique color.

Functionality:
- Reads .txt files containing electrode names, types, and 3D locations (X, Y, Z).
- Cleans and formats the data by removing unnecessary orientation and session columns.
- Aggregates electrode data across participants.
- Plots all electrode positions in a single 3D figure.
- Optionally displays electrode labels (e.g., Pz, AF7) at their respective locations.


In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

In [ ]:
dig_folder = 'D:\\BonoKat\\research project\\# study 1\\dig_data\\files for analysis\\Y'
eeg_data_dir = 'D:\\BonoKat\\research project\\# study 1\\eeg_data\\set\\Y'
subs_list = os.listdir(eeg_data_dir)

# Set the header line
column_names = ['Electrode Name', 'Electrode Type', 'Session Name', 'Loc. X', 'Loc. Y', 'Loc. Z',
                'm0n0', 'm0n1', 'm0n2', 'm1n0', 'm1n1', 'm1n2', 'm2n0', 'm2n1', 'm2n2']
cols_to_drop = ['m0n0', 'm0n1', 'm0n2', 'm1n0', 'm1n1', 'm1n2', 'm2n0', 'm2n1', 'm2n2']

def plot_electrodes_3d(dig_points_list, participant_labels, show_labels=True):
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    cmap = plt.cm.get_cmap('tab10', len(dig_points_list))

    for idx, dig_df in enumerate(dig_points_list):
        x = dig_df['Loc. X'].astype(float).values
        y = dig_df['Loc. Y'].astype(float).values
        z = dig_df['Loc. Z'].astype(float).values
        names = dig_df['Electrode Name'].values

        ax.scatter(x, y, z, color=cmap(idx), label=participant_labels[idx], s=40)

        if show_labels:
            for xi, yi, zi, label in zip(x, y, z, names):
                ax.text(xi, yi, zi, label, fontsize=8)

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.set_title('3D Electrode Positions Across Participants')
    ax.legend()
    plt.tight_layout()
    plt.show()

# --------- Main Script ---------
dig_points_list = []
participant_labels = []

for subject in subs_list[3:6]:
    dig_file = os.path.join(dig_folder, f"{subject}.txt")

    try:
        # Load and clean the data
        dig_points_df = pd.read_csv(dig_file, names=column_names, sep='\t', comment='#').iloc[:-4]
        dig_points_df.drop(columns=cols_to_drop + ['Session Name'], inplace=True)
        dig_points_df.dropna(inplace=True)

        dig_points_list.append(dig_points_df)
        participant_labels.append(subject)

    except Exception as e:
        print(f"Error processing {subject}: {e}")

# Plot all participants
plot_electrodes_3d(dig_points_list, participant_labels)



C:\Users\a1902989\AppData\Local\Temp\ipykernel_19972\2885497849.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = plt.cm.get_cmap('tab10', len(dig_points_list))


In [33]:
dig_points_list

[   Electrode Name Electrode Type   Loc. X    Loc. Y   Loc. Z
 0              P8            EEG  64.9684  -88.7053  10.7895
 1             FC2            EEG  19.8196   12.4283  91.6162
 2             CP6            EEG  72.8081  -62.4670  48.7149
 3              F5            EEG -64.7345   41.8276  28.8007
 4              P5            EEG -69.0268  -82.1362  28.7888
 ..            ...            ...      ...       ...      ...
 56            FPZ            EEG   2.9125   88.4519  11.6159
 57            TP8            EEG  78.3178  -62.4480   9.6880
 58             T8            EEG  86.4953  -32.1210  10.5779
 59            FCZ            EEG -10.6169   15.5153  93.2628
 60            PO4            EEG  20.5950 -107.4940  45.7120
 
 [61 rows x 5 columns],
    Electrode Name Electrode Type   Loc. X    Loc. Y   Loc. Z
 0              C2            EEG  36.3097    1.1006  88.8951
 1             FT7            EEG -80.4812   10.7183   1.1856
 2              F2            EEG  31.5182  

In [35]:
# Combine all electrode data across participants into a single DataFrame
# Add a 'Participant' column to trace origin
all_dfs = []

for df, label in zip(dig_points_list, participant_labels):
    df_copy = df.copy()
    df_copy['Participant'] = label
    all_dfs.append(df_copy)

combined_df = pd.concat(all_dfs, ignore_index=True)

# Group by electrode name
grouped = combined_df.groupby('Electrode Name')

# Compute descriptive stats for coordinates
desc_stats = grouped[['Loc. X', 'Loc. Y', 'Loc. Z']].agg(['mean', 'std', 'min', 'max', 'count'])

# Flatten the column MultiIndex
desc_stats.columns = ['_'.join(col).strip() for col in desc_stats.columns.values]

# Optional: sort by electrode name
desc_stats = desc_stats.sort_index()

# Show a preview
print(desc_stats[:])


                Loc. X_mean  Loc. X_std  Loc. X_min  Loc. X_max  Loc. X_count  \
Electrode Name                                                                  
AF3              -29.406767    6.343296    -35.4354    -22.7898             3   
AF4               36.943333    5.608264     31.3772     42.5928             3   
AF7              -53.009933    5.822341    -58.3784    -46.8208             3   
AF8               60.652567    1.820728     58.7509     62.3798             3   
C1               -29.346867    9.810302    -40.6702    -23.4048             3   
...                     ...         ...         ...         ...           ...   
PZ                -0.411800    9.885510    -11.7495      6.4041             3   
T7               -85.892467    0.934137    -86.4860    -84.8157             3   
T8                86.875533    0.470635     86.4953     87.4019             3   
TP7              -83.323133    1.849837    -85.4048    -81.8677             3   
TP8               80.019367 

In [38]:
desc_stats[:]

,Loc. X_mean,Loc. X_std,Loc. X_min,Loc. X_max,Loc. X_count,Loc. Y_mean,Loc. Y_std,Loc. Y_min,Loc. Y_max,Loc. Y_count,Loc. Z_mean,Loc. Z_std,Loc. Z_min,Loc. Z_max,Loc. Z_count
Electrode Name,,,,,,,,,,,,,,,
AF3,-29.406767,6.343296,-35.4354,-22.7898,3,76.173867,5.033968,70.9945,81.0486,3,31.800667,10.745334,19.4294,38.8085,3
AF4,36.943333,5.608264,31.3772,42.5928,3,71.742300,4.983439,68.6864,77.4929,3,33.156767,12.606829,21.2346,46.3517,3
AF7,-53.009933,5.822341,-58.3784,-46.8208,3,66.915767,5.410566,61.1083,71.8143,3,0.234600,8.088361,-8.6795,7.1053,3
AF8,60.652567,1.820728,58.7509,62.3798,3,58.601133,1.540215,56.8910,59.8791,3,-0.213700,12.136599,-10.7970,13.0336,3
C1,-29.346867,9.810302,-40.6702,-23.4048,3,-8.527500,10.127998,-18.5663,1.6874,3,94.754900,2.672806,92.0439,97.3878,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PZ,-0.411800,9.885510,-11.7495,6.4041,3,-80.239800,7.520883,-84.6749,-71.5561,3,86.770567,5.865130,82.8845,93.5171,3
T7,-85.892467,0.934137,-86.4860,-84.8157,3,-19.533700,5.050644,-23.2788,-13.7895,3,5.662867,10.592991,-6.0000,14.6872,3
T8,86.875533,0.470635,86.4953,87.4019,3,-29.559267,2.756511,-32.1210,-26.6424,3,1.768267,8.337867,-6.0000,10.5779,3
